# Realistic Cross-View Image Geo-Localization
_adjusted to GPU_


### Setup

In [1]:
# from google.colab import drive
# import os

# ROOT = '/content/gdrive'
# drive.mount(ROOT) # mount drive

# %cd "/content/gdrive/MyDrive/adl4cv/Realistic-Cross-View-Image-Geo-Localization/"

### Imports

In [1]:
# General packages
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from argparse import Namespace

# Modeling packages
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline
plt.rcParams['figure.figsize'] = (4.0, 3.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize

# for auto-reloading external modules see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Prepare data

In [4]:
# from data/convert_polar.py
############################ Apply Polar Transform to Aerial Images in CVUSA Dataset ############################
from data.convert_polar import polar_transform_CVUSA

# if os.listdir produces a timeout (colab does that after 2 min), then just try again and again (8-10 times even), it will work eventually
# https://stackoverflow.com/questions/54973331/input-output-error-while-using-google-colab-with-google-drive
# as suggested on stackoverflow, you can check the following command too
# !ls '../CVUSA/bingmap/19/' | grep "0044417"
# to see if a file can be found or not, if this does not work, probably os.listdir will not either

# takes approx. 4 hours
#polar_transform_CVUSA(input_dir = 'data/CVUSA/bingmap/19/', output_dir = 'data/CVUSA/polarmap/')



In [5]:
print(len(os.listdir('data/CVUSA/polarmap/')))
print(len(os.listdir('data/CVUSA/bingmap/19/')))
print(len(os.listdir('data/CVUSA/streetview/panos/')))

44416
44416
44516


### Train model 

In [2]:
# flush out allocated gpu memory
# as for some reason it "sticks" in if you tried to allocate too much
# factory reset runtime would be needed if this does not work
import gc
gc.collect()
torch.cuda.empty_cache()

# check allocated gpu memory
!nvidia-smi

Sat Dec 18 19:03:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070    Off  | 00000000:02:00.0 Off |                  N/A |
| 22%   45C    P5    12W / 180W |      0MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1070    Off  | 00000000:03:00.0 Off |                  N/A |
| 24%   

In [2]:
!python train.py

----------------- 
 Namespace(results_dir='./output', name='', seed=10, phase='train', gpu_ids='0,1', isTrain=True, resume=True, start_epoch=0, data_root='./data/CVUSA', train_csv='train-19zl-10.csv', val_csv='val-19zl-3.csv', polar=True, save_step=10, rgan_checkpoint=None, n_epochs=10, batch_size=24, lr_g=0.0001, lr_d=0.0001, lr_r=0.0001, b1=0.5, b2=0.999, lambda_gp=10, lambda_l1=100, lambda_ret1=1000, lambda_sm=10, hard_topk_ratio=1.0, hard_decay1_topk_ratio=0.1, hard_decay2_topk_ratio=0.05, hard_decay3_topk_ratio=0.01, n_critic=1, input_c=3, segout_c=3, realout_c=3, n_layers=3, feature_c=64, g_model='unet-skip', d_model='basic', r_model='SAFA', gan_loss='vanilla', lambda=10, condition=1)
----------------- Options ---------------
                       b1: 0.5                           
                       b2: 0.999                         
               batch_size: 24                            
                condition: 1                             
                  d_model:

In [ ]:
# check if an image can be found in our data folder
# if this throws a timeout (2min) than the dataloader probably won't work either
# try to run it until it works
!ls 'data/CVUSA/polarmap/' | grep "0041073"

0041073.jpg


### Test model

In [ ]:
!python test.py